# Utils Script



### This script consists of the different utility functions for the liteloan scorecard development 
### The Functions do a range of tasks from 

    - Computing PIR (Personal Indebtness ratio and FAI (Free Available Income) 
    - Compute other derived variables 
    - Plot graphs 
    - Create extensive analytical tables 
    

In [6]:
import warnings
import datetime as dt
warnings.filterwarnings("ignore")

import re
import string 
import copy

import numpy as np
import pandas as pd
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.compose import ColumnTransformer
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import RepeatedStratifiedKFold
# from sklearn.metrics import fbeta_score
# from sklearn.metrics import make_scorer
# from matplotlib import pyplot
# from sklearn.linear_model import LogisticRegression
# from imblearn.pipeline import Pipeline
# from imblearn.under_sampling import TomekLinks
# from imblearn.under_sampling import EditedNearestNeighbours
# from imblearn.under_sampling import RepeatedEditedNearestNeighbours
# from imblearn.under_sampling import NeighbourhoodCleaningRule
# from imblearn.under_sampling import OneSidedSelection
# from imblearn.under_sampling import InstanceHardnessThreshold
# from sklearn.linear_model import RidgeClassifier
# from imblearn.pipeline import Pipeline
# from imblearn.combine import SMOTEENN
# import statsmodels.api as sm
# from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import roc_auc_score
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    
# from sklearn.model_selection import train_test_split
# # from sklearn.linear_model import LogisticRegression
# # from sklearn.metrics import roc_auc_score

# pd.set_option('max_rows', None)
# pd.set_option('max_columns', None)
# pd.set_option('max_colwidth', None)
# pd.set_option('display.float_format', lambda x: '%.3f' % x)


import matplotlib.pyplot as plt
# from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
# plt.style.use('ggplot')
# %matplotlib inline
# import seaborn as sns
# color = sns.color_palette()



print("Util Libraries successfully imported")

Util Libraries successfully imported


In [7]:
pd.__version__

'1.4.2'

## Compute Free Available Income (FAI) function 

In [6]:
def calc_income(df, db, cr, opening_balance): 
    
    """
     %% parameters 

    df: [dataframe] - holding dataframe of income values 

    db: [str] - debit column 

    cr: [str]  - credit columns 

    opening_balance: [str] - opening balance column 
    
    
    ex: 
    
    calc_income('total_credit', 'total_debit', 'opening_balance')
    """
    
    def cred_deb_list(df):
        
      #  """
      #  + This function takes in the in dataframe 
      #  + returns list of credit months, debit months
      #  + returns as zipped or iterable list      
      #  """" 
        try:
            cred_month_list = [cred for cred in df.columns if cred.startswith(cr)]
            debit_month_list = [deb for deb in df.columns if deb.startswith(db)]
            month_list = list(zip(cred_month_list, debit_month_list))
        except error as e:
            print(e)
        finally:
            print('month lists have been extracted')
        
        return cred_month_list, debit_month_list, month_list 
    
    credit_month_list, debit_month_list, month_list = cred_deb_list(df)
    
    
    col_no = 1
    cred_count = 0
    deb_count = 0
    opening_balance = df[opening_balance]
    
    for cred, deb in month_list:
        
        if col_no <= 7:
            
            df["FAI_" + str(col_no)] = opening_balance + df[credit_month_list[cred_count]] - df[debit_month_list[deb_count]]
            opening_balance =   df["FAI_" + str(col_no)]
            col_no +=1
            cred_count += 1
            deb_count += 1
            
        else:
            
            break 
            
    return df 
                          




## Compute Personal Indebtnedness Ratio

+ PIR_1 = (𝑇𝑜𝑡𝑎𝑙 𝐼𝑛𝑠𝑡𝑎𝑙𝑙𝑚𝑒𝑛𝑡 𝐴𝑚𝑜𝑢𝑛𝑡 𝑃𝑎𝑖𝑑)/(Current FAI +𝑇𝑜𝑡𝑎𝑙 𝐼𝑛𝑠𝑡𝑎𝑙𝑙𝑚𝑒𝑛𝑡 𝐴𝑚𝑜𝑢𝑛𝑡 𝑃𝑎𝑖𝑑)  - Current working definition
+ PIR_2 = (𝑇𝑜𝑡𝑎𝑙 𝐼𝑛𝑠𝑡𝑎𝑙𝑙𝑚𝑛𝑡 𝐴𝑚𝑜𝑢𝑛𝑡 𝑃𝑎𝑖𝑑+𝑅𝑒𝑛𝑀𝑜𝑛𝑒𝑦 𝐼𝑛𝑠𝑡𝑎𝑙𝑙𝑚𝑒𝑛𝑡)/(𝐶𝑙𝑜𝑠𝑖𝑛𝑔 𝐵𝑎𝑙𝑎𝑛𝑐𝑒+𝑇𝑜𝑡𝑎𝑙 𝐼𝑛𝑠𝑡𝑎𝑙𝑙𝑚𝑒𝑛𝑡 𝐴𝑚𝑜𝑢𝑛𝑡 𝑃𝑎𝑖𝑑)
+ PIR_3 = (𝑇𝑜𝑡𝑎𝑙 𝑃𝑟𝑖𝑛𝑐𝑖𝑝𝑎𝑙 𝑅𝑒𝑐𝑒𝑖𝑣𝑒𝑑)/(𝐶𝑙𝑜𝑠𝑖𝑛𝑔 𝐵𝑎𝑙𝑎𝑛𝑐𝑒−𝑇𝑜𝑡𝑎𝑙 𝑃𝑟𝑖𝑛𝑐𝑖𝑝𝑎𝑙 𝑅𝑒𝑐𝑒𝑖𝑣𝑒𝑑)
+ PIR_4 = (𝑇𝑜𝑡𝑎𝑙 𝑃𝑟𝑖𝑛𝑐𝑖𝑝𝑎𝑙 𝑅𝑒𝑐𝑒𝑖𝑣𝑒𝑑+𝑃𝑟𝑖𝑛𝑐𝑖𝑝𝑎𝑙𝑅𝑒𝑛𝑀𝑜𝑛𝑒𝑦)/(𝐶𝑙𝑜𝑠𝑖𝑛𝑔 𝐵𝑎𝑙𝑎𝑛𝑐𝑒−𝑇𝑜𝑡𝑎𝑙 𝑃𝑟𝑖𝑛𝑐𝑖𝑝𝑎𝑙 𝑅𝑒𝑐𝑒𝑖𝑣𝑒𝑑(𝑁𝑜𝑛𝑅𝑒𝑛𝑚𝑜𝑛𝑒𝑦))

In [7]:
def calc_avg_std(df):
    FAI_cols = [col for col in df.columns if col.startswith("FAI")]
    df['FAI_Avg'] = df[FAI_cols].mean(axis=1)
    df['FAI_std'] = df[FAI_cols].std(axis=1)
    df['FAI_min_debit'] = df[debit_month_list].min(axis=1)
    df['FAI_max_debit'] = df[debit_month_list].max(axis=1)
    df['FAI_min_credit'] = df[credit_month_list].min(axis=1)
    df['FAI_max_credit'] = df[credit_month_list].max(axis=1)
    return df 

In [8]:
def compute_PIR(df, loan_repayment_col, loan_Id, closing_FAI, no_months):
    
    """
    
    df: [dataframe] -  holding dataframe of loan data 
    
    loan_repayment_col: [str] - column repayment name 
    
    loan_id: [str] - column ID 
    
    closing_FAI: [str] - closing Free Available Income 
    
    
    """
    
    #filter out repayment columns 
    
    repayment_cols = [col for col in df.columns if col.startswith(loan_repayment_col)]
    filtered_cols = repayment_cols + [loan_Id]
    repayment_df = df.filter(filtered_cols)
    
    #remove duplicate columns 
    dupl_cols = [col for col in df.columns if col in repayment_df.columns]
    dupl_cols.remove(loan_Id)
    
    to_merge_df = df.drop(dupl_cols, axis=1)
    
    #merge on Loan Id to comp_df
    
    repayment_df = to_merge_df.merge(repayment_df, on=loan_Id, how='outer')
    
   
    
    
    #compute on PIR_1 
    
    total_installment_paid = repayment_df[repayment_cols].sum(axis=1)
    closing_balance = repayment_df[closing_FAI]
    repayment_df['total_installment_paid'] = total_installment_paid 
    average_installment_paid = repayment_df['total_installment_paid'] / no_months
    repayment_df['average_installment_paid'] = average_installment_paid 
    repayment_df['PIR_1'] = average_installment_paid / (closing_balance + average_installment_paid)
    
    
    return repayment_df
    

In [9]:
# A function to calculate WOE and IV for numerical features
def woe_num(data, feature, gbflag):
    dt = pd.crosstab(index=data[feature], columns=data[gbflag])
    dt['Freq'] = dt.sum(axis=1)
    dt['Percentage'] = round((dt['Freq']/dt['Freq'].sum() *100),1)
    dt['% Good'] = round((dt['Good']/dt['Good'].sum() *100), 1)
    dt['% Bad'] = round((dt['Bad']/dt['Bad'].sum() *100), 1)
    dt['Bad Rate'] = round((dt['Bad']/dt['Freq']) *100, 1)
    dt['GoodBaddOdds'] = round(dt['Good']/dt['Bad'], 2)
    dt['WOE'] = np.log(dt['% Good']/dt['% Bad'])
    dt['IV'] = (dt['% Good']- dt['% Bad']) * dt['WOE']
#     dt['IV'] = ((dt['% Good']- dt['% Bad']) * dt['WOE']).sum()
   # dt = dt.sort_values(['WOE'])
    
    return dt
# Function to calculate WOE and IV for categorical variables
# Function to calculate WOE and IV for categorical variables
def woe_cat(data, feature, gbflag):
    dt = pd.crosstab(index=data[feature], columns=data[gbflag])
    dt['Freq'] = dt.sum(axis=1)
    dt['Proptn'] = dt['Freq']/dt['Freq'].sum()
    dt['% Good'] = dt['Good']/dt['Good'].sum() 
    dt['% Bad'] = dt['Bad']/dt['Bad'].sum()
    dt['Bad Rate'] = dt['Bad']/dt['Freq'] 
    dt['GoodBaddOdds'] = round(dt['Good']/dt['Bad'], 2)
    dt['WOE'] = np.log(dt['% Bad']/dt['% Good'])  # Ratio of Event to Non-Event
    dt['class IV'] = (dt['% Bad']-dt['% Good']) * dt['WOE']  # IV for each class/categpry
    dt['Variable IV'] = ((dt['% Bad']-dt['% Good']) * dt['WOE']).sum() #IV for the variable
    dt = dt.sort_values(['WOE'])
    
    return dt
# A function to get all variables WOE and IV
def woe_iv(df, variablie_list, GBFlag):   
    output = {}
    for variable in variablie_list:
        try:
            var = woe_cat(df, variable, GBFlag)
            
            for i in range(len(var.index.values)):
                output.setdefault('Variables',[]).append(variable)
                output.setdefault('Categories',[]).append(var.index.values[i])
                output.setdefault('WOE',[]).append(var['WOE'].values[i])
                output.setdefault('IV',[]).append(var['IV'].values[i])
        except Exception as e:
            pass

    return pd.DataFrame(output)
# woe_iv(data_universe_fai, data_universe_fai_cat.columns, 'GBFlag')
# A function that automates and return CA report
def Export_CA_Report(df, variablie_list, GBFlag, fileName='CA_Report'):
    
    # This inner function estimates WoE and IV which represent CA for all the provided variables
    def CA_Report(df, variablie_list, GBFlag):
        # Create a dictionary to track CA iteratively
        output = {}
        for variable in variablie_list:
            # use the function for estimating WoE for categorical variables to 
            var = woe_cat(df, variable, GBFlag)
            # append to dictionary
            output[variable]=var        

        return output
    
    
   # Instatiate above function to CA report for all provided variables
    ca_result = CA_Report(df, variablie_list, GBFlag)
    # save CA report to Excel
    with pd.ExcelWriter(fileName +'.xlsx', engine="openpyxl") as writer:
        for variable_name, data in ca_result.items():
            data.to_excel(writer, sheet_name=variable_name)
    
    
def fill_binned_nan_values(df, variablie_list, GBFlag):
    for variable in variablie_list:
        if df[variable].isna().sum() > 0:
            # Run CA(WoE & IV) on variable
            run_ca = woe_num(df, variable, GBFlag)
            # get index value(bin class) of category with least woe
            bin_class = run_ca[run_ca['WOE']==max(run_ca['WOE'])].index.values[0]
            df[variable] = df[variable].fillna(bin_class)
        else:
            pass

In [5]:
pd.__version__

'1.4.2'

In [10]:
def compute_FAI(df, total_credit_cols, total_debit_cols, opening_bal):
    # get all total credit columns
    cred_month_list = [cred for cred in df.columns if cred.startswith(total_credit_cols)]
    # get all total debit columns
    debit_month_list = [deb for deb in df.columns if deb.startswith(total_debit_cols)]
    # get initial opening balance
    opening_balance = df[opening_bal]
    # compute FAI_1 from opening balance
    df['FAI_1'] = opening_balance + df[cred_month_list[0]] - df[debit_month_list[0]]
    # compute subsequent FAIs from previous FAI values
    for i in range(1, len(cred_month_list)):
        df["FAI_" + str(i+1)] = df['FAI_'+str(i)] + df[cred_month_list[i]] - df[debit_month_list[i]]
        
    return df



# A function to quickly search for variables in large dataframe
def search_variable(df, variable):
    res =  [col for col in df.columns if variable.lower() in col.lower()]
    if res:
        return res
    return 'Not Found in DataFrame'

def convert_from_lower_to_upper(df1, list):
    
    '''
    function to map old ss variable name to new one for PSI
    '''
    
    res= [col for col in df1.columns]
    a = len(res)
    
    while a>0:
        for var in res:
            for var1 in list:
                if var == var1.lower():
                    df1.rename(columns = {var: var1}, inplace = True)
                    
        a-=1
        
    return df1
        
        
    
    

In [11]:
def missing_zero_values_table(df):
  
    """
    function for data audit, returns the count and percentage of missing and o values in each column
     %% parameters 

    df: [dataframe] 
    
    """
  
    zero_val = (df == 0.00).astype(int).sum(axis=0)
        
    zero_val_percent = (df == 0.00).astype(int).sum(axis=0) / len(df)
        
    mis_val = df.isnull().sum()
    
    mis_val_percent = round(df.isnull().sum() / len(df),1)
    
    mz_table = pd.concat([zero_val,zero_val_percent, mis_val, mis_val_percent], axis=1)
    
    mz_table = mz_table.rename(
                                columns = {0 : 'No of 0s', 1 : '% of 0s', 2: 'No of Missing Values', 3 : '% of Missing Values'})
    
    mz_table['Rows'] = len(df)
    mz_table['No of Unique'] = df.nunique()
    
    mz_table['Data Type'] = df.dtypes
    
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
        "There are " + str(mz_table.shape[0]) +
            " columns that have missing values.")
    
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
    return mz_table


def audit(data):
    
    result = {}
    
    for col in data.columns:
        No_unique = data[col].nunique()
        Missing = data[col].isna().sum()
        perc_missing = Missing/data.shape[0]
        Zeros = data[data[col] == 0].shape[0]
        perc_zeros = Zeros/data.shape[0]
        row_count = data[col].shape[0]
        
        result.setdefault('Variable', []).append(col)
        result.setdefault('No of Unique', []).append(No_unique)
        result.setdefault('No of Missing', []).append(Missing)
        result.setdefault('% Missing', []).append(perc_missing)
        result.setdefault('No of Zeros', []).append(Zeros)
        result.setdefault('% Zeros', []).append(perc_zeros)
        result.setdefault('No of Rows', []).append(row_count)
        
    res = pd.DataFrame(result)
    
    return res


In [12]:
from IPython.display import display_html


def display_side_by_side(*args):
    html_str = ''
    for df in args:
        html_str += df.to_html()
    display_html(html_str.replace(
        'table', 'table style="display:inline"'), raw=True)
    
    
def sample_first_prows(data, perc=0.8):
    '''function to get specified percentage of first rows in the dataframe
    '''
    return data.head(int(len(data)*(perc)))

# train = sample_first_prows(data)
# test = data.iloc[max(train.index):]


In [13]:
def fill_binned_nan_values(df, variablie_list, GBFlag):
    for variable in variablie_list:
        ## Coarse NaN group into the worst woe group if not more than 10% of total data shape
      
            # Run CA(WoE & IV) on variable using the woe_num function
            run_ca = woe_num(df, variable, GBFlag)
            # get index value(bin class) of category with worst woe category
            bin_class = run_ca[run_ca['WOE']==max(run_ca['WOE'])].index.values[0]
            df[variable] = df[variable].fillna(bin_class)
            
       

In [14]:
def replace_with_woe(df_with_selected_variables):
    '''
    This function replaces each category in the selected columns with
    their woe
    
    df_with_selected_variables :: Dataframe containing grouped/binned selected
    variables
    
    grpd_variables_woe_iv :: Dataframe containing WoE and IV of grouped/binned
    variables
    
    Note:: Make sure woe_iv function has returned it's result before running
    this function. result in this case - grpd_variables_woe_iv
    
    '''
    
    for variable in df_with_selected_variables.columns:
        for category in df_with_selected_variables[variable].unique():
            
            try:
                ### Get WoE from the dataframe running the WoE_IV function
                woe =variables_woe_iv[
                    (variables_woe_iv['Variables']==variable) & 
                    (variables_woe_iv['Categories']==category)
                ]['WOE'].values[0]

                ### Replace each category with their respective woe
                df_with_selected_variables[variable] = df_with_selected_variables[variable].replace(category, woe)
            except Exception as e:
                pass
    
    return df_with_selected_variables

In [15]:
# # A function to get all variables WOE and IV
# def woe_iv(df, variablie_list, GBFlag):   
#     output = {}
#     for variable in variablie_list:
#         try:
#             var = woe_cat(df, variable, GBFlag)
            
#             for i in range(len(var.index.values)):
#                 output.setdefault('Variables',[]).append(variable)
#                 output.setdefault('Categories',[]).append(var.index.values[i])
#                 output.setdefault('WOE',[]).append(var['WOE'].values[i])
#                 output.setdefault('Variable IV',[]).append(var['Variable IV'].values[i])
#         except Exception as e:
#             pass

#     return pd.DataFrame(output)






# A function to get all variables WOE and IV
def woe_iv(df, variablie_list, GBFlag):   
    output = {}
    for variable in variablie_list:
        try:
            var =  (pd.crosstab(df[variable],df[GBFlag],normalize='columns').assign(woe=lambda dfx: np.log(dfx['Bad'] / dfx['Good'])).assign(iv=lambda dfx: np.sum(dfx['woe']*(dfx['Bad']-dfx['Good']))))
            
            for i in range(len(var.index.values)):
                output.setdefault('Variables',[]).append(variable)
                output.setdefault('Categories',[]).append(var.index.values[i])
                output.setdefault('WOE',[]).append(var['woe'].values[i])
                output.setdefault('Variable IV',[]).append(var['iv'].values[i])
        except Exception as e:
            pass

    return pd.DataFrame(output)

def iv(df, variablie_list, GBFlag):   
    output = {}
    for variable in variablie_list:
        try:
            var = woe_cat(df, variable, GBFlag)
            
            for i in range(len(var.index.values)):
                output.setdefault('Variables',[]).append(variable)
                output.setdefault('Variable IV',[]).append(var['Variable IV'].values[i])
        except Exception as e:
            pass

    return pd.DataFrame(output)

##### undersampling

In [16]:
## modeling

## undersampling

# calculate f2-measure
def f2_measure(y_true, y_pred):
	return fbeta_score(y_true, y_pred, beta=2)
 
# evaluate a model
def evaluate_model(X, y, model):
	# define evaluation procedure
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	# define the model evaluation metric
	metric = make_scorer(f2_measure)
	# evaluate model
	scores = cross_val_score(model, X, y, scoring=metric, cv=cv, n_jobs=-1)
	return scores
 
# define undersampling models to test
def get_models():
	models, names = list(), list()
	# TL
	models.append(TomekLinks())
	names.append('TL')
	# ENN
	models.append(EditedNearestNeighbours())
	names.append('ENN')
	# RENN
	models.append(RepeatedEditedNearestNeighbours())
	names.append('RENN')
	# OSS
	models.append(OneSidedSelection())
	names.append('OSS')
	# NCR
	models.append(NeighbourhoodCleaningRule())
	names.append('NCR')
  #InstanceHardnessThreshold

	models.append(InstanceHardnessThreshold())
	names.append('IHT')
	return models, names
 

In [6]:
### This function helps in computing Gini from AUC


#https://stackoverflow.com/questions/66679372/how-does-the-predict-function-of-statsmodels-interact-with-roc-auc-score-of-scik


from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
def compute_gini(y_test, y_train, test_preds, train_preds):
    # instantiate roc auc score from sci-kit learn metrics and compute AUC

    Test_AUC = roc_auc_score(y_test, test_preds)
    Train_AUC = roc_auc_score(y_train, train_preds)
    # Compute Gini from AUC
    Test_Gini = (2*Test_AUC) - 1
    Train_Gini = (2*Train_AUC) - 1
    return Test_Gini, Train_Gini

In [ ]:
def compute_recall(y_test, y_train, test_preds, train_preds):
    # instantiate roc auc score from sci-kit learn metrics and compute AUC
    train_preds = train_preds.to_list()
    train_preds = np.asarray(train_preds)
    test_preds = test_preds.to_list()
    test_preds = np.asarray(test_preds)
    Test_precision, Test_recall, thresh = precision_recall_curve(y_test, test_preds)
    Train_precision, Train_recall, train_thresh = precision_recall_curve(y_train, train_preds)

    return Test_recall, Train_recall

In [ ]:
def compute_precision(y_test, y_train, test_preds, train_preds):
    # instantiate roc auc score from sci-kit learn metrics and compute AUC
    train_preds = train_preds.to_list()
    train_preds = np.asarray(train_preds)
    test_preds = test_preds.to_list()
    test_preds = np.asarray(test_preds)
    Test_precision, Test_recall, thresh = precision_recall_curve(y_test, test_preds)
    Train_precision, Train_recall, train_thresh = precision_recall_curve(y_train, train_preds)

    return Test_precision, Train_precision

In [5]:
# import numpy as np
# y_true = np.array([0, 0, 1, 1])
# y_scores = np.array([0.1, 0.4, 0.35, 0.8])
# precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
# precision

array([0.66666667, 0.5       , 1.        , 1.        ])

#### Gini test

In [ ]:
# def univariate_gini_computation(df, features, target):

    
#     # create an empty dictionary
#     gini_ = {}
    
#     # iteratively get the features from a list of features
#     for feature in features:
#         # extrac the feature and target from the entire dataframe
#         data = df[[feature, target]]
#         # encode the target variable; Good-0, Bad-1
#         data[target] = np.where(data[target]=='Bad', 1, 0)
#         # get dummies for the feature (independent variable)
#         data = pd.concat([data, pd.get_dummies(data[feature], prefix=feature, prefix_sep='-')],axis=1)
#         # drop the feature since we have its dummies
#         data = data.drop(feature, axis=1)
        
#         # Split data for training
#         X = data.drop(target, axis=1).values
#         y = data[target].values
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        
#         # instantiate logistic regression algo
#         lr = LogisticRegression()
#         # fit data to the algorithm
#         lr.fit(X_train, y_train)
#         # predict probability of default (PD) from test data
#         y_pred = lr.predict_proba(X_test)
#         # extract the PD
#         y_pred = y_pred[:][:, 1]
        
#         # Estimate AUC
#         AUC = roc_auc_score(y_test, y_pred)
#         # Compute Gini from AUC
#         Gini = 2*AUC - 1
        
#         # Append both feature and Gini to the above dictionary
#         gini_.setdefault('Feature',[]).append(feature)
#         gini_.setdefault('Gini',[]).append(Gini)
        
#         # All result to be returned in a dataframe
#         result = pd.DataFrame(gini_)

#     return result

In [ ]:
def plot_by_woe(df_WoE, rotation_of_x_axis_labels = 0):
    x = np.array(df_WoE.index).astype(str)
    y = df_WoE['WOE']
    plt.figure(figsize = (26, 8))
    plt.plot(x, y, marker = 'o', linestyle = '--', color = 'k')
    #plt.xlabel(x)
    plt.ylabel('Weight of Evidence')
    plt.title(str(''))
    plt.xticks(rotation = rotation_of_x_axis_labels)

In [ ]:
def check_if_equal(data):
  result = {}
  result['Variable1'] = []
  result['Variable2'] = []
  var = data.columns.to_list()
  for i in range(len(var)-1):
    for col in var:
      if (data[var[i]].equals(data[col])) & (var[i]!=col):
        if col in result['Variable2']:
          pass
        else:
         result['Variable1'].append(col)
         result['Variable2'].append(var[i])
         
         
  result = pd.DataFrame(data = result)
  return result